### Laboratorio 6 - Sistema de Recomendación (Basado En Filtro Colaborativo)
Stefano Aragoni, Carol Arévalo

-----------

En esta práctica se diseñó una red neuronal para la predicción de ratings de películas, basado en el filtro colaborativo. Dicha red recibe el ID de un usuario y el ISBN de una película, y el grupo de usuario al que pertenece, y devuelve un rating estimado para dicha película.

A continuación se muestra el código utilizado para la creación de la Red Neuronal, así como los resultados obtenidos.

------- 

##### *Importar librerías*

Como primer paso, se importaron las librerías necesarias para el desarrollo del laboratorio.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, GaussianDropout
from keras import regularizers
from keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU
from keras.layers import Activation
from keras.callbacks import EarlyStopping

##### *Cargar datos*

Posteriormente, se cargaron los datos del archivo ./archive/final_ratings.csv. Este ya fue analizado y procesado en el análisis exploratorio de datos, por lo que no se realizaron modificaciones adicionales.

In [ ]:
# Cargamos los datos
df = pd.read_csv('./archive/final_ratings.csv')

df.head()

,User-ID,Location,Age,ISBN,Book-Author,Year-Of-Publication,Publisher,Book-Rating,Continent,User-Group
0,248805,111,34.0,425118703.0,21432,1995.0,1714,9,3,17
1,167614,111,46.0,345337662.0,5191,1993.0,1388,6,3,26
2,140036,111,35.0,688149553.0,81432,1997.0,11425,9,3,31
3,50446,111,18.0,679734775.0,86489,1991.0,15646,5,3,46
4,265889,18,30.0,743444477.0,12279,2003.0,11710,9,3,36


##### *Conjunto de Entrenamiento y Validación*

Se separó el conjunto de datos en un conjunto de entrenamiento y un conjunto de validación, con el fin de poder evaluar el desempeño de la red neuronal.

In [ ]:
scaler = MinMaxScaler()

X = df[['User-ID', 'User-Group', 'ISBN']].values
X = scaler.fit_transform(X)

y = df[['Book-Rating']].values
y = scaler.fit_transform(y)

# Dividimos los datos en entrenamiento y validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(243676, 3) (243676, 1) (60919, 3) (60919, 1)


##### *Diseño de Red Neuronal*

Posteriormente, se diseñó la red neuronal. Esta recibe como entrada el ID de un usuario, el ISBN de una película, y el grupo de usuario al que pertenece, y devuelve un rating estimado para dicha película.

Esta red permite un Sistema de Recomendación basado en Filtro Colaborativo, ya que permite predecir el rating que un usuario le daría a una película, basado en los ratings que otros usuarios le han dado a dicha película y sus similitudes con el usuario en cuestión.

In [ ]:
model2 = Sequential()

model2.add(Dense(256, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('relu'))
model2.add(BatchNormalization())

model2.add(Dense(128, kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('relu'))
model2.add(Dropout(0.3))

model2.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('relu'))
model2.add(BatchNormalization())

model2.add(Dense(32, kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('relu'))
model2.add(Dropout(0.3))

model2.add(Dense(16, kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('relu'))
model2.add(BatchNormalization())

model2.add(Dense(1, activation='sigmoid'))


model2.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history2 = model2.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[early_stopping])

print(f"Training loss (Model 1): {history2.history['loss'][-1]} \n")
loss1 = model2.evaluate(X_test, y_test)
print(f"\nTest loss (Model 1): {loss1}")

Epoch 1/20
476/476 [==============================] - 5s 7ms/step - loss: 0.5069 - val_loss: 0.1092
Epoch 2/20
476/476 [==============================] - 3s 6ms/step - loss: 0.1058 - val_loss: 0.1066
Epoch 3/20
476/476 [==============================] - 3s 6ms/step - loss: 0.1054 - val_loss: 0.1066
Epoch 4/20
476/476 [==============================] - 3s 6ms/step - loss: 0.1053 - val_loss: 0.1066
Epoch 5/20
476/476 [==============================] - 3s 6ms/step - loss: 0.1053 - val_loss: 0.1066
Epoch 6/20
109/476 [=====>........................] - ETA: 2s - loss: 0.1047

KeyboardInterrupt: 